<a href="https://colab.research.google.com/github/pcadmanbosse/cs224u/blob/main/bnb_4bit_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [3]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:

from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[ "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 21260288 || all params: 3773331456 || trainable%: 0.5634354746703705


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [61]:
from datasets import Dataset
import pandas as pd
train = pd.read_pickle("../content/train_dataset.pkl")
def create_training_entry(row):
    return "<s>[INST]"+row["QUERY"]+" Document: " + row["TEXT"] + "Scorecard: "+",".join(row["SCORECARD"])+"[INST] \n"+row["RESULT"]+"</s>"

index = range(0, len(train.index))
train["text"] = train.apply(create_training_entry, axis=1)
train["input_ids"] = index
training_ds = Dataset.from_pandas(train)

KeyError: ignored

In [62]:
train[["text", "input_ids"]]

,text,input_ids
95,<s>[INST]Which organic fertilizers are present...,0
478,<s>[INST]Which prohibited pesticides are prese...,1
111,<s>[INST]Which organic fertilizers are present...,2
372,<s>[INST]Which prohibited pesticides are prese...,3
141,<s>[INST]Which organic fertilizers are present...,4
...,...,...
41,<s>[INST]Which organic fertilizers are present...,545
254,<s>[INST]Which organic fertilizers are present...,546
441,<s>[INST]Which prohibited pesticides are prese...,547
234,<s>[INST]Which organic fertilizers are present...,548


In [41]:
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 13.8 MB/s eta 0:00:00


Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [72]:
import transformers
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "garbage_collection_threshold:0.9,max_split_size_mb:512"
torch.cuda.empty_cache()

tokenizer.pad_token = tokenizer.eos_token
training_arguments = transformers.TrainingArguments(
        remove_unused_columns = False,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    )

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)
trainer = SFTTrainer(
    model=model,
    train_dataset=training_ds,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4048,  # You can specify the maximum sequence length here
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Map:   0%|          | 0/550 [00:00<?, ? examples/s]

OutOfMemoryError: ignored

In [48]:
train.iloc[0].text

"<s>[INST]Which organic fertilizers are present on which plots? here is the document: REGISTRO DE APLICAÇÃO DE INSUMOS\nSAFRA 2022/2023\nVICENCA\nProprietário: Vicente Ferreira Gonçalves Fazenda Escalada\nCAFÊS ESPECIAIS\nATIVIDADE: PULVERIZAÇÃO FOLIAR\nTALHÕES: 16, 17, 18, 19, 20, 21 = 57,82 ha. (Obs: Aplicação em 25% da área)\nRec.:\n052/23\nNutrição Foliar, Prevenção e Controle de Cercosporiose, Phoma, Lagartas e Bicho mineiro\nJustificativa/Alvo:\nMarcha Trator:\n1ª B\nRotação:\n1800 RPM\nTempo/50m:\n27 S\n500 ml\nTotal de Bicos:\n4\nVazão/Bico:\nBico Usado:\nCone vazio\nQuantidade Coletada:\n2,0 Litros\nPulverizador:\nArbus\nGolden\n2000 L\nIntervalo de Segurança:\nEntrada Lavoura:\n2 dias\n30 dias\nDose para\nVazão por\nInsumo\nIngrediente ativo\nDose/ha (Kg/Lt)\nQtde Total\nAlvo do Produto aplicado\n2000 Lts\nha\nÁgua\nH2O\n384,00\n1920,0 Lt\n5550,72\nQuimifol P30W\nN - 1%; P205 - 30,0%; Mg 1,5%\n1,5\n7,5\n21,68\nNutrição\nCarbonato de Sódio 3,0% Carbonato de\nVeeper\n1,0\n5,0 L

In [60]:
eval_prompt = """Which organic fertilizers are present on which plots? here is the document: REGISTRO DE APLICAÇÃO DE INSUMOS
SAFRA 2022/2023
VICENCA
Proprietário: Vicente Ferreira Gonçalves Fazenda Escalada
CAFÊS ESPECIAIS
ATIVIDADE: PULVERIZAÇÃO FOLIAR
TALHÕES: 16, 17, 18, 19, 20, 21 = 57,82 ha. (Obs: Aplicação em 25% da área)
Rec.:
052/23
Nutrição Foliar, Prevenção e Controle de Cercosporiose, Phoma, Lagartas e Bicho mineiro
Justificativa/Alvo:
Marcha Trator:
1ª B
Rotação:
1800 RPM
Tempo/50m:
27 S
500 ml
Total de Bicos:
4
Vazão/Bico:
Bico Usado:
Cone vazio
Quantidade Coletada:
2,0 Litros
Pulverizador:
Arbus
Golden
2000 L
Intervalo de Segurança:
Entrada Lavoura:
2 dias
30 dias
Dose para
Vazão por
Insumo
Ingrediente ativo
Dose/ha (Kg/Lt)
Qtde Total
Alvo do Produto aplicado
2000 Lts
ha
Água
H2O
384,00
1920,0 Lt
5550,72
Quimifol P30W
N - 1%; P205 - 30,0%; Mg 1,5%
1,5
7,5
21,68
Nutrição
Carbonato de Sódio 3,0% Carbonato de
Veeper
1,0
5,0 Lt
Condicionador de calda
Potassio 1,0%
14,46
(AZOXISTROBINA) 200 g/L;
Priori Top
DIFENOCONAZOL 125 g/L
0,4
2,0 Lt
Cercosporiose e Phoma sp
5,78
Cercobin 875 WG
(TIOFANATO-METÍLICO) 125 g/Kg
0,7
3,5 Kg
10,12
Cercosporiose
(CLORANTRANILIPROLE) 45 g/L;
Voliam Targo
(ABAMECTINA) 18 g/L
0,5
2,5 Lt
7,23
Bicho mineiro
Ochima
Ésteres alquílicos do ácido fosfórico
0,4
2,0 Lt
5,78
Adjuvante
400 LT
Quimifol Calcio e Boro Il
Ca 8,0% B 2,0%;
2,0
10,0 Lt
28,91
Nutrição
Heavy
N 22,0%
5,0
25,0 Lt
72,28
Nutrição
N 5,0% P205 8,0%; K20 5,0%;
Niphokam
Mg-1,0%;B-0,5%;Cu-0,2%; Mn -
2,0
10,0 Lt
28,91
Nutrição
0,5%; Zn 1,0%
Raynitro
Co 0,5%; Mo 5,0%
2,0
10,0 Lt
28,91
Nutrição
Mg 5,5%; 10,0% 5,0% Cu 0,5%; Fe
Quimifol SQB
3,0
15,0 Kg
43,37
Nutrição
0,1%; Mo 0,1%; 5,0%
Quimifol K-40
N 10,0%; K20 40,0%
2,0
10,0 Kg
28.91
Nutrição
Açúcar
C12H24O12
4,0
20,0 Kg
57,82
Glicose
Obs: Nas aplicações de defensivos: Evitar períodos mais quentes do dia, com ventos fortes e/ ou chuvosos. Não aplicar
defensivos dentro de 10 m de APP's, Reserva Legal e/ou vegetação nativa. A sobra da mistura deverá ser aplicada na borda-
dura da lavoura ou no talhão subsequente. Caso seja o último talhão, o mesmo deverá ser aplicado na sua bordadura não
ultrapassando a dosagem máxima aplicada.
Quantidade:
Local: ( ) Bordadura Talhão subsequente.
06/03/2023
Reginaldo André de Oliveira
Eng° Agrônomo
CREA: MG 211634/D
and here is the scorecard: Quimifol K-40,Ca 8,0% B 2,0%;,N 22,0%,Quimifol Calcio e Boro Il, casca de pinus e eucalipto e farelos vegetais, Turfa,Fertilizante de origem vegetal e animal,MicroControl, Sulfato de Manganês, Melaço de cana-de- açúcar,Biomix Mudas & Plantio Orgânico,H2O,Fertilizante Orgânico Simples Classe A - Openeem Flex,Excrementos de animais, compostos e biofertilizantes,EOX 710 MAG, Ácido cítrico(conservante),Niphokam, Termofosfato,Fertilizante Orgânico Simples Classe A - Adubos Ouro,Melaço de cana-de- açúcar,Farinha de penas,Heavy,(AZOXISTROBINA) 200 g/L; DIFENOCONAZOL 125 g/L,Voliam Targo,Palha de café,Produtos, subprodutos e resíduos industriais de origem vegetal, Farelo de cereal,Extratos botânicos deAzadirachta indica,MZE Biorganic,Hidrolisado de peixe (pescado marinho), melaço decana-de-açúcar, sulfato de potássio, ácido cítrico esacarídeos.,N 5,0% P205 8,0%; K20 5,0%; Mg-1,0%;B-0,5%;Cu-0,2%; Mn - 0,5%; Zn 1,0%,(CLORANTRANILIPROLE) 45 g/L; (ABAMECTINA) 18 g/L,Turfa, Aminoácido, Agente alcalinizante,(TIOFANATO-METÍLICO) 125 g/Kg,Quimifol P30W,Fertilizante Orgânico Simples Classe A - Openeem Mix, Fibra de coco,Fertilizante Orgânico Niorg (farelado e granulado),N - 1%; P205 - 30,0%; Mg 1,5%, Casca de madeira,Carbonato de Sódio 3,0% Carbonato de Potassio 1,0%, Sacarídeos,AO-15, Carcaça de peixe, Micronutrientes,Cinzas,Casca de pinus,Veeper,Fertilizante Orgânico Composto Classe A - Brutal Plus,Ochima, Calcário dolomítico,Fertilizante foliar à base de alga Ascophyllum nodosun e extrator Hidróxido de Potássio,Visafértil - Torta de Mamona,Estercos,Produtos e subprodutos processados de origem animal,Esterco e camas de aviário, Casca de Pinus/Eucalipto e Tortas e farelos vegetais,Torta de mamona PauliVida,Extrato hidroalcoólico de compostos botânicos de Azadirachta indica,HYT-C MaisRaiz,Raynitro,FishFértil Classic,Açúcar, Chá,Quimifol SQB, Esterco bovino / suíno / codorna,Co 0,5%; Mo 5,0%, Resíduos de origem vegetal, Farinha de ossos,Fertilizante Orgânico Composto Classe A - Pra Terra, Cinzas,Água,Ésteres alquílicos do ácido fosfórico,Matéria orgânica de origem vegetal e animal,Aloe Plus,Torta de mamona,Composto fermentado biológico-enzimático,FertPremium Blend G8,Hidrolisado de peixe (pescados marinhos), melaço de cana-de- açúcar, Ácido Cítrico e Sacarídeos,Cama de aviário,Resíduos de origem vegetal,Fertilizante Orgânico Composto Classe A - Visafértil Origem,Cercobin 875 WG, Matéria seca - biomassa,N 10,0%; K20 40,0%,Biomix Biokashi,Fertilizante Orgânico Composto Classe A - Forte C, Melaço de cana- de-açúcar,Mg 5,5%; 10,0% 5,0% Cu 0,5%; Fe 0,1%; Mo 0,1%; 5,0%,Magmaton,Extrato glicólico de Aloe vera ,Extrato de Algas Acadian,C12H24O12, Leite descartado / gordura de leite,Esterco de frango"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
model.eval()
with torch.no_grad():
    generated_code = tokenizer.decode(model.generate(**model_input, max_new_tokens=50, pad_token_id=2)[0], skip_special_tokens=True)
print(generated_code)

Which organic fertilizers are present on which plots? here is the document: REGISTRO DE APLICAÇÃO DE INSUMOS
SAFRA 2022/2023
VICENCA
Proprietário: Vicente Ferreira Gonçalves Fazenda Escalada
CAFÊS ESPECIAIS
ATIVIDADE: PULVERIZAÇÃO FOLIAR
TALHÕES: 16, 17, 18, 19, 20, 21 = 57,82 ha. (Obs: Aplicação em 25% da área)
Rec.:
052/23
Nutrição Foliar, Prevenção e Controle de Cercosporiose, Phoma, Lagartas e Bicho mineiro
Justificativa/Alvo:
Marcha Trator:
1ª B
Rotação:
1800 RPM
Tempo/50m:
27 S
500 ml
Total de Bicos:
4
Vazão/Bico:
Bico Usado:
Cone vazio
Quantidade Coletada:
2,0 Litros
Pulverizador:
Arbus
Golden
2000 L
Intervalo de Segurança:
Entrada Lavoura:
2 dias
30 dias
Dose para
Vazão por
Insumo
Ingrediente ativo
Dose/ha (Kg/Lt)
Qtde Total
Alvo do Produto aplicado
2000 Lts
ha
Água
H2O
384,00
1920,0 Lt
5550,72
Quimifol P30W
N - 1%; P205 - 30,0%; Mg 1,5%
1,5
7,5
21,68
Nutrição
Carbonato de Sódio 3,0% Carbonato de
Veeper
1,0
5,0 Lt
Condicionador de calda
Potassio 1,0%
14,46
(AZOXISTROBINA) 200 g